In [472]:
import numpy as np
import scipy.linalg
#import cudaq

import poly
import sym_qsp_opt
 
import matplotlib.pyplot as plt

In [473]:
# Classical Linear System Solver
class ClassicalSolver:
    def __init__(self, A, b):
        self.A = A
        self.b = b

    def solve(self):
        """
        Solves the linear system Ax = b using a classical solver.
        
        Returns:
            x: Solution vector.
        """
        x = scipy.linalg.solve(self.A, self.b)  # Using SciPy's solver for Ax = b
        return x

In [474]:
class QuantumSolverQSVT:
    def __init__(self, A, b, epsilon=1e-1, phases=None):
        """
        Initializes the QuantumSolverQSVT with matrix A and vector b.
        
        Parameters:
            A (numpy.ndarray): The coefficient matrix for the linear system.
            b (numpy.ndarray): The right-hand side vector of the linear system.
        """
        self.A = A
        self.b = b
        # TODO: alpha and kappa may need to be replaced by their respective upper bounds
        self.alpha = np.linalg.norm(A, ord=2)  # Spectral norm
        self.eigenvalues, self.eigenvetor= np.linalg.eig(A)
        self.A_rescaled = self.A / self.alpha
        self.kappa = np.linalg.cond(A) # Condition number
        self.b_norm = np.linalg.norm(b) # Normalization 
        self.epsilon = epsilon # Desired presision
        self.N = len(b)
        self.n = int(np.log2(self.N))
        self.m = 1 # Number of ancillas for the blocj-encoding of A
        self.f = 1 #flags where to apply Pi

        if phases is None:
        # if False:
            pg = poly.PolyOneOverX()
            pcoefs, scale = pg.generate(kappa=self.kappa, 
                                        epsilon=self.epsilon, 
                                        chebyshev_basis=True, 
                                        return_scale=True)
            # Using odd parity and instantiating desired coefficeints.
            parity = 1
            coef = pcoefs[parity::2]
            # Optimize to the desired function using Newton solver.
            crit = 1e-12
            (phases, err, total_iter, qsp_seq_opt) = sym_qsp_opt.newton_solver(coef, parity, crit=crit)
        self.phases = phases    
        self.d = len(self.phases) - 1

    def is_proper_quantum_state(self, vector):
        """
        Check if a vector is a valid quantum state.
        
        Parameters:
            vector (numpy.ndarray): The vector to check.
            
        Returns:
            bool: True if the vector is a valid quantum state, False otherwise.
        """
        # Ensure the input is a NumPy array
        vector = np.array(vector, dtype=np.complex128)
        
        # Check normalization
        norm = np.linalg.norm(vector)  # Calculate the L2 norm
        is_normalized = np.isclose(norm, 1.0)
        
        # Ensure it's not an empty vector
        is_nonempty = vector.size > 0

        return is_nonempty and is_normalized

    def _block_encode_A(self):
        """
        Creates a block-encoded unitary matrix U that embeds A / alpha in its top-left block.
        """
        I = np.eye(self.A_rescaled.shape[0])
        top_right_block = scipy.linalg.sqrtm(I - self.A_rescaled @ self.A_rescaled.T.conj())
        bottom_left_block = scipy.linalg.sqrtm(I - self.A_rescaled.T.conj() @ self.A_rescaled)

        U = np.block([[self.A_rescaled, top_right_block],
                      [bottom_left_block, -1*self.A_rescaled.T.conj()]])
        U_T = U.T
        return U, U_T

    def _state_preparation_b(self):
        """
        Creates a state-preparation unitary matrix Q s.t. Q|0> = |b> / ||b>|
        """
        B = np.column_stack((self.b / self.b_norm, np.random.randn(self.N, self.N - 1)))
        # Apply QR decomposition to B to get an orthonormal basis
        # The Q matrix from the QR decomposition will be unitary, and the first column will be b
        Q, _ = scipy.linalg.qr(B, mode='economic')
        return Q
    
    def _P(self, phi):
        dim = int(2**(self.m+self.f))
        P = np.diag([np.exp(1j * phi), np.exp(-1j * phi)] * (dim //2))
        #P = np.diag([np.exp(1j * phi)] + [np.exp(-1j * phi)] * (dim - 1))
        return P

    def _build_circuit(self):

        H_gate = 1/np.sqrt(2)*np.array([[1, 1], [1, -1]])

        num_qubits = self.n + self.m + self.f

        initial_state_circuit = np.concatenate(([1], np.zeros(2**num_qubits-1)))
        #print("initial state")
        #print(initial_state_circuit)

        U_b=self._state_preparation_b()

        circuit=initial_state_circuit @ np.kron( np.eye(2**(self.m + self.f)), U_b)
        #print("prepared b state")
        #print(circuit)
        #print(self.is_proper_quantum_state(circuit))
        
        U_A, U_A_T = self._block_encode_A()
        
        
        circuit= circuit @ np.kron(H_gate,np.eye(2**(self.n + self.m)))
        #print("H gate")
        #print(circuit)
        #print(self.is_proper_quantum_state(circuit))

        circuit = circuit @ np.kron(self._P(self.phases[0]), np.eye(2**(self.n)))
        #print("Pi")
        #print(circuit)
        #print(self.is_proper_quantum_state(circuit))

        i=0
        for phi in reversed(self.phases[:-1]):

            if i%2==0:
                circuit = circuit @ np.kron( np.eye(2**(self.f)), U_A)
                #print(circuit)
                #print(self.is_proper_quantum_state(circuit))
            else:
                circuit = circuit @ np.kron( np.eye(2**(self.f)), U_A_T)
                #print(circuit)
                #print(self.is_proper_quantum_state(circuit))

            circuit = circuit @ np.kron(self._P(phi), np.eye(2**(self.n)))
            #print(circuit)
            #print(self.is_proper_quantum_state(circuit))

            i=i+1
        
        circuit= circuit @ np.kron(H_gate,np.eye(2**(self.n + self.m)))
        print(self.is_proper_quantum_state(circuit))
        print("circuit =",circuit)

        #readout the f, m register, they must be in state 0f 0m,
        #then the result is in the n register
        readout = np.zeros(2)

        readout[0] = np.sqrt(sum([((np.absolute(circuit[i]))**2) for i in range(0,8,2)]))
        readout[1] = np.sqrt(sum([((np.absolute(circuit[i]))**2) for i in range(1,8,2)]))

        #readout=circuit[0:self.f+1]
        print(self.is_proper_quantum_state(readout))
        print("readout =",readout)

        #rescale the result
        print("alpha=",self.alpha)
        rescaled=self.b_norm*(1/self.alpha)*readout#*(1/min(self.eigenvalues))
        #rescaled=(1)*readout
        
        return rescaled
    
    def solve(self):
        """
        Solves the system Ax = b using a QSVT-based inversion (placeholder).
        
        Returns:
            x (numpy.ndarray): Solution vector, as if from a QSVT inversion.
        """
        x = 0
        return x

In [475]:
# # Define a test matrix A and vector b
A = np.array([[1    ,0 ],
              [  0  ,1/6 ]], dtype=float)
b = np.array([1,1], dtype=float)
solver = QuantumSolverQSVT(A,b)

b=147, j0=35
[PolyOneOverX] minimum [-7.75401864] is at [-0.09251427]: normalizing
[sym_qsp] Iterative optimization to err 1.000e-12 or max_iter 1000.
iter: 001 --- err: 1.891e-01
iter: 002 --- err: 2.469e-02
iter: 003 --- err: 9.102e-04
iter: 004 --- err: 1.488e-06
iter: 005 --- err: 4.041e-12
iter: 006 --- err: 3.428e-15
[sym_qsp] Stop criteria satisfied.


In [476]:
A_inverse_b=solver._build_circuit()
print("result")
print((A_inverse_b))

True
circuit = [-0.69649348-0.12205259j -0.33524917-0.61559269j  0.        +0.j
 -0.0050699 +0.09288664j  0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j        ]
True
readout = [0.70710678 0.70710678]
alpha= 1.0
result
[1. 1.]


In [477]:
A_inverse_b_classical = scipy.linalg.solve(A,b)
print(A_inverse_b_classical)

[1. 6.]


In [478]:
distance_solutions= np.linalg.norm(A_inverse_b - A_inverse_b_classical)
print(distance_solutions)

4.999999999999999


In [479]:
# shift and scale the spectrum to be in [dist, pi-dist]
val_H, vec_H = la.eigh(H)
val_H = val_H.real
val_H_min = val_H.min()
val_H_max = val_H.max()
grd_state = vec_H[:, val_H.argmin()]

c1 = (np.pi-2*dist) / (val_H_max - val_H_min)
c2 = dist - c1 * val_H_min
   
val_H_trans = val_H * c1 + c2
    
t_tot = tau * c1
shift = tau * c2

NameError: name 'la' is not defined